# Simulate AWS DMS output

In my thesis, the input files are pretended to be written by AWS DMS. Instead of actually creating a relational database and extracting it using AWS DMS, I have chosen to directly create the output files.

In [52]:
import pandas as pd
import random
import os
from helpers.paths import PathMerger
from faker import Faker
from datetime import datetime, timedelta, date, timezone
from dateutil import tz
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

## Functions

In [53]:
for i in range(9):
    print(i)

0
1
2
3
4
5
6
7
8


In [54]:
def generate_customers(first_registration:datetime):
    customers = []
    
    # Memorized value for the creation time of the previous customer.
    created_prev = None
    
    # A single hard-coded customer for reference with an id of 0
    customer = {
        'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'id': 1, 
        'username': 'janisourander@kamk.fi',
        # 'password': fake.md5(),
        'created': datetime(1970, 1, 15, 10, 0, 0),
        'modified': datetime(1970, 2, 20, 12, 34, 56),
    }
    
    customers.append(customer)

    for i in range(9):
        
        if created_prev is None:
            # The first ever customer was created at 13:37, 15th of Jan 2020.
            created = first_registration
        else:
            # Each other customer were created n second later than the previous.
            created = created_prev + timedelta(seconds=random.randrange(1, 3600))
        
        customer = {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': len(customers), 
            'username': fake.unique.ascii_email(),
             # 'password': fake.md5(),
            'created': created,
            'modified': created + timedelta(seconds=random.randrange(0, 3600 * 24 * 365)),
        }
        
        customers.append(customer)
        
        # Set current created datetime as previous
        created_prev = created

    return pd.DataFrame.from_records(customers)
    
    
def generate_device_models():
    devices = [
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 1, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Red',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 1),
            'modified': datetime(2010, 3, 21, 12, 0, 1),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 2, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Black',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 2),
            'modified': datetime(2010, 3, 21, 12, 0, 2),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 3, 
            'release_date': date(2010, 11, 1),
            'name': 'Super Gadget 100',
            'color': 'Pink',
            'description': 'lorem ipsum',
            'created': datetime(2010, 8, 5, 7, 0, 0),
            'modified': datetime(2010, 8, 5, 7, 0, 0),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 4, 
            'release_date': date(2018, 5, 13),
            'name': 'Super Gadget 200',
            'color': 'White',
            'description': 'lorem ipsum',
            'created': datetime(2018, 3, 20, 12, 1, 1),
            'modified': datetime(2018, 3, 20, 12, 1, 1),
        },
    ]
    
    return pd.DataFrame.from_records(devices)


def generate_devices(customers: pd.DataFrame, devices: pd.DataFrame):

    # List of unique model numbers
    unique_device_models = list(devices.id.unique())
    
    # Container
    devices = []
    # i = 0

    for (_, cust) in customers.iterrows():
        
        # Fetch the creation date of user and add some
        created = cust.created + timedelta(seconds=random.randrange(1, 3600))
        
        # Create 1 or 2 devices per user by random
        for x in range(random.randrange(1, 3)):
            device = {
                'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'id': len(devices) + 1, 
                'customer_fk': cust.id,
                'model_fk': random.choice(unique_device_models),
                'serial_number': fake.ssn(),
                'created': created,
                'modified': created + timedelta(seconds=random.randrange(1, 3600)),
            }

            devices.append(device)
    
    return pd.DataFrame.from_records(devices)

        
def generate_customer_details(customers: pd.DataFrame):
    
    # Container
    details = []
    
    for (_, cust) in customers.iterrows():
        detail = {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': len(details) + 1,
            'customer_fk': cust.id,
            'birthday': fake.date_of_birth(),
            'language': fake.language_code(),
            'street_address': fake.street_address(),
            'postal_code': fake.postalcode(),
            'city': fake.city(),
            'country': fake.country_code(),
            'phone_number': fake.phone_number(),
            'created': cust.created,
            'modified': cust.created,
        }
        
        details.append(detail)
        
    return pd.DataFrame.from_records(details)

## Create Directories

In [55]:
# Table paths
table_paths = {}

# Map of combinations to create: (db, table, data_generator_function)
tables = [
    ('customers', 'customers'), 
    ('customers', 'customer_details'),
    ('devices', 'device_models'),
    ('devices', 'devices')
]

for db, table in tables:
    dt = f'{db}.{table}'
    sp = PathMerger(db, table).staging
    dp = os.path.join(sp, 'LOAD00000001.parquet')
    
    # Create dirs
    if not os.path.exists(sp):
        os.makedirs(sp)
        
    table_paths[dt] = dp

## Generate Datasets

### Customers

In [56]:
# Generate
df_c = generate_customers(first_registration=datetime(2020, 1, 15, 13, 37, 0))

# Write
df_c.to_parquet(table_paths['customers.customers'])

# Show
df_c

,dms_timestamp,id,username,created,modified
0,2021-08-05 15:19:54,1,janisourander@kamk.fi,1970-01-15 10:00:00,1970-02-20 12:34:56
1,2021-08-05 15:19:54,1,gabrielmills@gmail.com,2020-01-15 13:37:00,2021-01-07 09:15:05
2,2021-08-05 15:19:54,2,joseweber@gmail.com,2020-01-15 13:50:08,2020-05-15 17:43:29
3,2021-08-05 15:19:54,3,kjensen@wade.com,2020-01-15 14:11:15,2020-02-17 17:12:01
4,2021-08-05 15:19:54,4,alexandrawebb@gmail.com,2020-01-15 15:11:12,2020-07-02 01:06:48
5,2021-08-05 15:19:54,5,juliecrawford@delgado-adams.com,2020-01-15 15:36:15,2020-09-12 05:02:53
6,2021-08-05 15:19:54,6,thompsonrachel@morales-brown.com,2020-01-15 16:18:10,2020-09-19 08:25:01
7,2021-08-05 15:19:54,7,diane74@yahoo.com,2020-01-15 16:42:02,2020-12-23 11:11:17
8,2021-08-05 15:19:54,8,kimsusan@bradley.info,2020-01-15 17:20:06,2020-07-24 15:52:01
9,2021-08-05 15:19:54,9,ricardo84@norton.com,2020-01-15 18:18:34,2020-03-24 21:07:23


### Device Models

In [57]:
# Generate
df_dm = generate_device_models()

# Write
df_dm.to_parquet(table_paths['devices.device_models'])

# Show
df_dm

,dms_timestamp,id,release_date,name,color,description,created,modified
0,2021-08-05 15:19:54,1,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01
1,2021-08-05 15:19:54,2,2010-05-15,Super Gadget 100,Black,lorem ipsum,2010-03-21 12:00:02,2010-03-21 12:00:02
2,2021-08-05 15:19:54,3,2010-11-01,Super Gadget 100,Pink,lorem ipsum,2010-08-05 07:00:00,2010-08-05 07:00:00
3,2021-08-05 15:19:54,4,2018-05-13,Super Gadget 200,White,lorem ipsum,2018-03-20 12:01:01,2018-03-20 12:01:01


### Devices

In [58]:
# Generate devices based on customers and device models
df_d = generate_devices(df_c, df_dm)

# Write
df_d.to_parquet(table_paths['devices.devices'])

# Show
df_d

,dms_timestamp,id,customer_fk,model_fk,serial_number,created,modified
0,2021-08-05 15:19:54,1,1,1,862-86-8047,1970-01-15 10:18:54,1970-01-15 10:52:11
1,2021-08-05 15:19:54,2,1,1,329-08-2350,1970-01-15 10:18:54,1970-01-15 10:50:41
2,2021-08-05 15:19:54,3,1,3,360-73-1379,2020-01-15 14:04:02,2020-01-15 14:47:19
3,2021-08-05 15:19:54,4,1,3,034-94-0243,2020-01-15 14:04:02,2020-01-15 14:57:24
4,2021-08-05 15:19:54,5,2,1,688-21-1124,2020-01-15 14:10:11,2020-01-15 14:45:38
5,2021-08-05 15:19:54,6,2,2,531-52-1018,2020-01-15 14:10:11,2020-01-15 14:59:34
6,2021-08-05 15:19:54,7,3,4,225-91-3334,2020-01-15 14:14:20,2020-01-15 14:59:47
7,2021-08-05 15:19:54,8,4,2,339-08-2633,2020-01-15 16:06:51,2020-01-15 16:06:53
8,2021-08-05 15:19:54,9,5,3,839-26-4038,2020-01-15 16:04:49,2020-01-15 16:36:36
9,2021-08-05 15:19:54,10,5,1,624-20-4847,2020-01-15 16:04:49,2020-01-15 16:11:13


### Customer_details

In [59]:
# Generate customer details. One per customer.
df_cd = generate_customer_details(df_c)

# Write
df_cd.to_parquet(table_paths['customers.customer_details'])

df_cd

,dms_timestamp,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,2021-08-05 15:19:54,1,1,1963-12-23,ff,817 Robbins Parkway Suite 056,46333,North Calvin,GW,001-250-991-3804x31652,1970-01-15 10:00:00,1970-01-15 10:00:00
1,2021-08-05 15:19:54,2,1,1976-09-19,nso,433 Brooks Island Suite 741,43049,Michaelhaven,EE,464-893-6394x1626,2020-01-15 13:37:00,2020-01-15 13:37:00
2,2021-08-05 15:19:54,3,2,1970-03-30,hsb,87669 Corey Cliffs Apt. 675,91948,Stevensville,DM,+1-475-923-1883x1368,2020-01-15 13:50:08,2020-01-15 13:50:08
3,2021-08-05 15:19:54,4,3,1975-01-07,gez,8766 Kenneth Route Apt. 484,86596,Port Tamara,UY,790-317-9466x699,2020-01-15 14:11:15,2020-01-15 14:11:15
4,2021-08-05 15:19:54,5,4,2010-07-05,ps,0545 Morrow Valleys,17678,Hicksville,GD,374-161-5610x0808,2020-01-15 15:11:12,2020-01-15 15:11:12
5,2021-08-05 15:19:54,6,5,2007-02-03,am,398 William Divide Suite 196,77377,East Kari,TT,821-855-0428x92087,2020-01-15 15:36:15,2020-01-15 15:36:15
6,2021-08-05 15:19:54,7,6,1932-12-26,lt,2729 Shawna Tunnel,87032,East Jennifershire,KN,710.352.7405,2020-01-15 16:18:10,2020-01-15 16:18:10
7,2021-08-05 15:19:54,8,7,2014-05-03,mt,4647 Shannon Mountain,66937,Riddlefurt,IL,031-818-5508x0042,2020-01-15 16:42:02,2020-01-15 16:42:02
8,2021-08-05 15:19:54,9,8,1988-04-24,iu,593 Oneill Loop Apt. 187,73969,Port Amberport,BY,001-348-098-6291x67428,2020-01-15 17:20:06,2020-01-15 17:20:06
9,2021-08-05 15:19:54,10,9,2018-09-20,lg,911 Crawford Expressway,62185,New Kimfort,CZ,(989)386-1440,2020-01-15 18:18:34,2020-01-15 18:18:34


## List resulting files

In [60]:
# Maximum lenght of a path
max_w = max([len(x[1]) for x in table_paths.items()]) + 2

# UNIX epoch start
epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)

for k, f in table_paths.items():

    s = os.stat(f)
    f_path = '.' + os.sep + f 
    f_size = s.st_size // 1024 
    f_mod = (
        (epoch + timedelta(seconds=s.st_mtime))
        .astimezone(tz.gettz('Europe/Helsinki'))
        .strftime('%y-%m-%d %H:%M:%S')
    )
    
    print(f"{f_mod} | {f_path:<{max_w}} | {f_size:>2d} KB")

21-08-05 15:19:54 | .\S3\staging\dms\abc\customers\customers\LOAD00000001.parquet        |  4 KB
21-08-05 15:19:54 | .\S3\staging\dms\abc\customers\customer_details\LOAD00000001.parquet |  9 KB
21-08-05 15:19:54 | .\S3\staging\dms\abc\devices\device_models\LOAD00000001.parquet      |  6 KB
21-08-05 15:19:54 | .\S3\staging\dms\abc\devices\devices\LOAD00000001.parquet            |  6 KB


## Preview only first lines

This is for screenshot purposes.

In [61]:
from IPython.display import HTML

panda_tables = [df_c, df_cd, df_dm, df_d]

for pt in panda_tables:
    display(HTML(pt.head(1).to_html(index=False)))

dms_timestamp,id,username,created,modified
2021-08-05 15:19:54,1,janisourander@kamk.fi,1970-01-15 10:00:00,1970-02-20 12:34:56


dms_timestamp,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
2021-08-05 15:19:54,1,1,1963-12-23,ff,817 Robbins Parkway Suite 056,46333,North Calvin,GW,001-250-991-3804x31652,1970-01-15 10:00:00,1970-01-15 10:00:00


dms_timestamp,id,release_date,name,color,description,created,modified
2021-08-05 15:19:54,1,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01


dms_timestamp,id,customer_fk,model_fk,serial_number,created,modified
2021-08-05 15:19:54,1,1,1,862-86-8047,1970-01-15 10:18:54,1970-01-15 10:52:11
